Use this dataset of airline arrival information to predict how late flights will be. A flight only counts as late if it is more than 30 minutes late.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as sklearn_pca
from locale import atof
from sklearn import decomposition
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import ensemble

In [ ]:
# Read and import data
airlines = pd.read_csv('Airlines 2008.csv')
airlines.head()

In [ ]:
#Change the ArrDelay variable into a categorical variable
airlines.loc[airlines['ArrDelay'] <= 30, 'ArrDelay'] = 0
airlines.loc[airlines['ArrDelay'] > 30, 'ArrDelay'] = 1

In [ ]:
#Check length of the dataset
len(airlines)

In [ ]:
#Type of info per column of the dataset
airlines.info()

In [ ]:
# Compute average number of delayed flights per month
grouped = airlines[['ArrDelay', 'DayofMonth']].groupby('DayofMonth').mean()

# plot average delays by month
grouped.plot(kind='bar')

In [ ]:
# Compute average number of delayed flights per month
grouped = airlines[['ArrDelay', 'DayOfWeek']].groupby('DayOfWeek').mean()

# plot average delays by month
grouped.plot(kind='bar')

In [ ]:
# Compute average number of delayed flights per month
airlines['hour'] = airlines['CRSArrTime'].map(lambda x: int(str(int(x)).zfill(4)[:2]))
grouped = airlines[['ArrDelay', 'hour' ]].groupby('hour').mean()

# plot average delays by month
grouped.plot(kind='bar')

In [ ]:
# Compute average number of delayed flights per month
grouped = airlines[['ArrDelay', 'Month']].groupby('Month').mean()

# plot average delays by month
grouped.plot(kind='bar')

In [ ]:
# Compute average number of delayed flights per month
grouped = airlines[['ArrDelay', 'Month']].groupby('Month').mean()

# plot average delays by month
grouped.plot(kind='bar')

In [ ]:
#Check the number of Nan in the Dataframe
airlines.isnull().sum()

In [ ]:
#Drop columns that are not going to be used
airlines1 = airlines.drop(airlines[['Year','Month','UniqueCarrier','FlightNum',
                                    'TailNum','Origin','Dest',
                                    'CancellationCode',
                                   'CarrierDelay',
                                   'WeatherDelay',
                                   'NASDelay',
                                   'SecurityDelay',
                                   'LateAircraftDelay']],axis=1)

In [ ]:
#Clean columns that have NAn
airlines2=airlines1.drop(airlines1[['DepTime','ActualElapsedTime',
                                    'CRSElapsedTime','AirTime',
                                    'DepDelay','TaxiIn','TaxiOut','AirTime','Cancelled','Diverted'
                                    ]],axis=1)

airlines2.info()

In [ ]:
#Check the values that still have NAN
airlines2.isnull().sum()

In [ ]:
#Drop all columns with NAN
airlines3 = airlines2.dropna(how='any') 
airlines3.isnull().sum()

In [ ]:
#Check the new dataset
airlines3.head()

In [ ]:
#Count number of datapoints for ArrDelay
airlines3['ArrDelay'].value_counts()

In [ ]:
#Downsample majority class

# Separate majority and minority classes
airlines_majority = airlines3[airlines3.ArrDelay==0]
airlines_minority = airlines3[airlines3.ArrDelay==1]
 
# Downsample mairlinesass
airlines_majority_downsampled = resample(airlines_majority, replace=False, n_samples=901398, random_state=123) 
 
# Combine minority class with downsampled majority class
airlines_downsampled = pd.concat([airlines_majority_downsampled, airlines_minority])
 
# Display new class counts
airlines_downsampled.ArrDelay.value_counts()

In [ ]:
#Define Outcome & Predictors

y = airlines_downsampled['ArrDelay']
X = airlines_downsampled.drop('ArrDelay',axis=1)

#Scale the data
names = X.columns
X = pd.DataFrame(preprocessing.scale(X), columns = names)

#Split into test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#KFOld
kf = KFold(5)

In [ ]:
#PCA Analysis

In [ ]:
#PReprocess and scale data
names = X.columns
X_scaled = pd.DataFrame(preprocessing.scale(X), columns = names)
summary = X_scaled.describe()

In [ ]:
#PCA Analysis

# Create a scaler object
sc = StandardScaler()

# Fit the scaler to the features and transform
X_std = sc.fit_transform(X_scaled)

# Create a PCA object with the 30 components as a parameter
pca = decomposition.PCA(n_components=7)

# Fit the PCA and transform the data
X_std_pca = pca.fit_transform(X_std)

# View the new feature data's shape
X_std_pca.shape

# Create a new dataframe with the new features

X1 = pd.DataFrame(X_std_pca)
X1.head()


***Logistic Regression***

In [ ]:
# Initialize and fit the model.
lr = LogisticRegression()
fittrain = lr.fit(X_train,y_train)
fittest = lr.fit(X_test,y_test)

# Predict on training set
predtrain_y = lr.predict(X_train)
predtest_y = lr.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

#Training Scores
target_names = ['0.0', '1.0']
print(classification_report(y_train, predtrain_y, target_names=target_names))
cnf = confusion_matrix(y_train, predtrain_y)
print(cnf)

# Accuracy tables.
table_train = pd.crosstab(y_train, predtrain_y, margins=True)

train_tI_errors = table_train.loc[0.0,1.0] / table_train.loc['All','All']
train_tII_errors = table_train.loc[1.0,0.0] / table_train.loc['All','All']

print((
    'Training set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}\n\n'
).format(train_tI_errors, train_tII_errors))

In [ ]:
#Testing Scores
target_names = ['0.0', '1.0']
print(classification_report(y_test, predtest_y, target_names=target_names))
cnf = confusion_matrix(y_test, predtest_y)
print(cnf)

table_test = pd.crosstab(y_test, predtest_y, margins=True)

test_tI_errors = table_test.loc[0.0,1.0]/table_test.loc['All','All']
test_tII_errors = table_test.loc[1.0,0.0]/table_test.loc['All','All']

print((
    'Test set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}'
).format(test_tI_errors, test_tII_errors))
print(cross_val_score(lr,X,y,cv=kf))
print(cross_val_score(lr,X,y,cv=kf).mean())
print('Cross validation PCA:', cross_val_score(lr,X1,y,cv=kf).mean())


*** Random Forest***

In [ ]:
# Train model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.fit(X_test, y_test)

# Predict on training set
predtrainrf_y = rf.predict(X_train)
predtestrf_y = rf.predict(X_test)

In [ ]:
#Training Scores
target_names = ['0', '1']
print(classification_report(y_train, predtrainrf_y, target_names=target_names))
cnf = confusion_matrix(y_train, predtrainrf_y)
print(cnf)

# Accuracy tables.
table_train = pd.crosstab(y_train, predtrainrf_y, margins=True)

train_tI_errors = table_train.loc[0.0,1.0] / table_train.loc['All','All']
train_tII_errors = table_train.loc[1.0,0.0] / table_train.loc['All','All']

print((
    'Training set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}\n\n'
).format(train_tI_errors, train_tII_errors))

In [ ]:
#Test Scores
target_names = ['0', '1']
print(classification_report(y_test, predtestrf_y, target_names=target_names))
cnf = confusion_matrix(y_test, predtestrf_y)
print(cnf)

table_test = pd.crosstab(y_test, predtestrf_y, margins=True)

test_tI_errors = table_test.loc[0.0,1.0]/table_test.loc['All','All']
test_tII_errors = table_test.loc[1.0,0.0]/table_test.loc['All','All']

print((
    'Test set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}'
).format(test_tI_errors, test_tII_errors))
print(cross_val_score(rf,X,y,cv=kf))
print(cross_val_score(rf,X,y,cv=kf).mean())
print('Cross validation PCA:', cross_val_score(lr,X1,y,cv=kf).mean())


***SVC***

In [ ]:
# Train and fit the model
clf1 = SVC(kernel='linear', 
            class_weight='balanced',
            probability=True)
 
clf1.fit(X_train, y_train)
clf1.fit(X_test,y_test)
 
# Predict on training set
predtrainclf_y = clf1.predict(X_train)
predtestclf_y = clf1.predict(X_test)

In [ ]:
#Training Scores
target_names = ['0.0', '1.0']
print(classification_report(y_train, predtrainclf_y, target_names=target_names))
cnf = confusion_matrix(y_train, predtrainclf_y)
print(cnf)

# Accuracy tables.
table_train = pd.crosstab(y_train, predtrainclf_y, margins=True)

train_tI_errors = table_train.loc[0.0,1.0] / table_train.loc['All','All']
train_tII_errors = table_train.loc[1.0,0.0] / table_train.loc['All','All']

print((
    'Training set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}\n\n'
).format(train_tI_errors, train_tII_errors))

In [ ]:
#Testing Scores
target_names = ['0.0', '1.0']
print(classification_report(y_test, predtestclf_y, target_names=target_names))
cnf = confusion_matrix(y_test, predtestclf_y)
print(cnf)

table_test = pd.crosstab(y_test, predtestclf_y, margins=True)

test_tI_errors = table_test.loc[0.0,1.0]/table_test.loc['All','All']
test_tII_errors = table_test.loc[1.0,0.0]/table_test.loc['All','All']

print((
    'Test set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}'
).format(test_tI_errors, test_tII_errors))
print(cross_val_score(clf1,X,y,cv=kf))
print(cross_val_score(clf1,X,y,cv=kf).mean())
print('Cross validation PCA:', cross_val_score(lr,X1,y,cv=kf).mean())


***Gradient Boosting Classifier***

In [ ]:
# We'll make 500 iterations, use 2-deep trees, and set our loss function.
params = {'n_estimators': 500,
          'max_depth': 2,
          'loss': 'deviance'}

# Initialize and fit the model.
clf = ensemble.GradientBoostingClassifier(**params)
clf.fit(X_train, y_train)
clf.fit(X_test, y_test)

# Predict on training set
predtrainclf_y = clf.predict(X_train)
predtestclf_y = clf.predict(X_test)

In [ ]:
#Training Scores
target_names = ['0.0', '1.0']
print(classification_report(y_train, predtrainclf_y, target_names=target_names))
cnf = confusion_matrix(y_train, predtrainclf_y)
print(cnf)

# Accuracy tables.
table_train = pd.crosstab(y_train, predtrainclf_y, margins=True)

train_tI_errors = table_train.loc[0.0,1.0] / table_train.loc['All','All']
train_tII_errors = table_train.loc[1.0,0.0] / table_train.loc['All','All']

print((
    'Training set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}\n\n'
).format(train_tI_errors, train_tII_errors))

In [ ]:
#Test Scores
target_names = ['0.0', '1.0']
print(classification_report(y_test, predtestclf_y, target_names=target_names))
cnf = confusion_matrix(y_test, predtestclf_y)
print(cnf)

table_test = pd.crosstab(y_test, predtestclf_y, margins=True)

test_tI_errors = table_test.loc[0.0,1.0]/table_test.loc['All','All']
test_tII_errors = table_test.loc[1.0,0.0]/table_test.loc['All','All']

print((
    'Test set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}'
).format(test_tI_errors, test_tII_errors))
print(cross_val_score(clf,X,y,cv=kf))
print(cross_val_score(clf,X,y,cv=kf).mean())
print('Cross validation PCA:', cross_val_score(lr,X1,y,cv=kf).mean())

From the information we have about arilines and the delay they have experienced, we transform the delay from minutes into a categorical variable: 0 if th delay on arrival is less than 30 min and 1 if it is euqal or more.
When cleaning the dataset, all the missing datapoints have been taken out (more than 9m entries in total) without significantly impacting the dataset.
The dataset has been downsampled to the delays so that both classes are balanced when training different models. It has been downsampled considering the delays to reduce the computational power required.
In the first pass, all remaining features have been applied after testing through PCA the max accuracy that the model can achieve.
From all the models that have been tried, the one that best fits the data is the Gradient Boosting Classifier followed by the Random Forest one. Although both require a igh computational power, Random Forest require less.
